In [1]:
# packages
import pandas as pd
from mod02_build_bot_predictor import train_model

### Define a function to extract predictions from the model

In [2]:
def predict_bot(df, model=None):
    """
    Predict whether each account is a bot (1) or human (0).
    """
    if model is None:
        model = train_model()

    preds = model.predict(df)
    return pd.Series(preds, index=df.index)

### Define a function to evaluate model error

In [3]:
def confusion_matrix_and_metrics(y_true, y_pred):
    """
    Computes confusion matrix and common error rates for binary classification.

    Assumes labels:
      0 = negative class
      1 = positive class

    Returns:
      dict with:
        tn, fp, fn, tp
        misclassification_rate
        false_positive_rate
        false_negative_rate
    """
    tn = fp = fn = tp = 0

    for yt, yp in zip(y_true, y_pred):
        if yt == 0 and yp == 0:
            tn += 1
        elif yt == 0 and yp == 1:
            fp += 1
        elif yt == 1 and yp == 0:
            fn += 1
        elif yt == 1 and yp == 1:
            tp += 1
        else:
            raise ValueError("Labels must be 0 or 1")

    total = tn + fp + fn + tp

    misclassification_rate = (fp + fn) / total if total > 0 else 0.0
    false_positive_rate = fp / (fp + tn) if (fp + tn) > 0 else 0.0
    false_negative_rate = fn / (fn + tp) if (fn + tp) > 0 else 0.0

    return {
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn,
        "misclassification_rate": misclassification_rate,
        "false_positive_rate": false_positive_rate,
        "false_negative_rate": false_negative_rate,
    }


### Load the data

In [4]:
TRAIN_PATH = "mod02_data/train.csv"
train = pd.read_csv(TRAIN_PATH)

TEST_PATH = "mod02_data/test.csv"
test = pd.read_csv(TEST_PATH)

### Format the data by independent vs. dependent variables

In [5]:
X_train = train.drop(columns=["is_bot"])
y_train = train['is_bot']

X_test = test.drop(columns=["is_bot"])
y_test = test['is_bot']

### Build the model on training data

In [6]:
model = train_model(X_train, y_train)

### Get the model predictions on training and test data

In [7]:
y_pred_train = predict_bot(X_train, model)
y_pred_test = predict_bot(X_test, model)

### Check results on the training set (data used to build the model)

In [8]:
confusion_matrix_and_metrics(y_train, y_pred_train)

{'tp': 171,
 'tn': 2611,
 'fp': 26,
 'fn': 192,
 'misclassification_rate': 0.07266666666666667,
 'false_positive_rate': 0.009859689040576413,
 'false_negative_rate': 0.5289256198347108}

### Check results on the test set (new data not yet seen by the model)

In [9]:
confusion_matrix_and_metrics(y_test, y_pred_test)

{'tp': 36,
 'tn': 846,
 'fp': 28,
 'fn': 90,
 'misclassification_rate': 0.118,
 'false_positive_rate': 0.032036613272311214,
 'false_negative_rate': 0.7142857142857143}

# Discussion Questions

### Based on the misclassification rate of your model, discuss your confidence in the ability to predict a bot. 

Based on the model's misclassification rate of 11.8%, I have decent confidence in it's ability to predict bots. The model performs fairly well, but it makes some mistakes such as missing certain bots. Because of this, it would be useful as a screening tool but shouldn't be solely and heavilty relied on to identify bots.

### What are potential ramifications of false positives from the model?

False positives happen when a real person is incorrectly labeled a bot. This could result when humans being restricted, flagged, or even banned. This can potentially cause reputational or financial harm if accounts are incorrectly flagged or restricted.

### What are potential ramifications of false negatives from the model?

False negatives happen when a bot is classified as a person. This allows bots to remain active which can lead to spam, and misinformation. These erros can be especially probelmatic because they allow harmful or deceptive activity to continue undetected.